In [79]:
#Plan
#json input parsing, extract snippet, comments
#prompt engineering: Ask for all features asked by the assignment, 
#Give full context(code, comment, purpose), 
#enforce format
# Output: positive paraphrasing, the Why, suggested improvement (snippet)
#add tooling to search for documentation links
# enforce tone change based on the severity of the review
# concluding paragraph to summarize feedback encouragingly
# few-shot examples for better results

# Creating a langchain agent along with prompt engineering

In [82]:
!pip install langchain-groq langchain-community duckduckgo-search

C:\Users\prudh\.conda\envs\kitti_det_env\Lib\site-packages\IPython\utils\_process_win32.py:138: ResourceWarning: unclosed file <_io.BufferedWriter name=3>
  res = process_handler(cmd, _system_body)
C:\Users\prudh\.conda\envs\kitti_det_env\Lib\site-packages\IPython\utils\_process_win32.py:138: ResourceWarning: unclosed file <_io.BufferedReader name=4>
  res = process_handler(cmd, _system_body)
C:\Users\prudh\.conda\envs\kitti_det_env\Lib\site-packages\IPython\utils\_process_win32.py:138: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  res = process_handler(cmd, _system_body)


In [1]:
from langchain_groq import ChatGroq
from langchain.tools import Tool
from langchain.agents import create_react_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate
from langchain_community.tools import DuckDuckGoSearchRun
import json
import re

### Load LLM

In [4]:
llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0.7,
    max_tokens=3000,
    api_key=""
)

### Create Search Tool

In [7]:
search_tool = DuckDuckGoSearchRun()

def create_documentation_search_tool():
    """Create tool for finding relevant documentation links"""
    
    def search_documentation(query: str) -> str:
        """Search for programming documentation and best practices"""
        # Enhanced search queries for better results
        enhanced_queries = [
            f"{query} official documentation",
            f"{query} best practices programming",
            f"{query} code style guide"
        ]
        
        results = []
        for enhanced_query in enhanced_queries[:2]:  # Limit for speed
            try:
                result = search_tool.run(enhanced_query)
                results.append(result)
            except:
                continue
                
        return "\n".join(results)
    
    return Tool(
        name="documentation_search",
        description="Search for relevant programming documentation, style guides, and best practices",
        func=search_documentation
    )

documentation_tool = create_documentation_search_tool()

### Prompts

In [10]:
SEVERITY_CLASSIFICATION_PROMPT = """
Analyze the severity of these code review comments and classify each one:

CODE CONTEXT:
{code_snippet}

COMMENTS TO CLASSIFY:
{comments_list}

For each comment, classify as:
- CRITICAL: Security issues, major bugs, or performance problems
- MODERATE: Code quality, maintainability, or readability issues  
- MINOR: Style preferences, naming conventions, or minor optimizations

Return in format:
Comment 1: [SEVERITY] - [BRIEF_REASON]
Comment 2: [SEVERITY] - [BRIEF_REASON]
...
"""

In [12]:
FEW_SHOT_EXAMPLES = """
EXAMPLE TRANSFORMATIONS:

Original: "This is inefficient. Don't loop twice conceptually."
Severity: MODERATE - Performance optimization opportunity

Positive Rephrasing: "Great logic flow here! I can see you're thinking through the filtering process step by step. For even better performance, especially with larger datasets, we can streamline this approach."

The 'Why': List iterations can become expensive with large datasets. Python's list comprehensions are not just more concise—they're actually optimized at the C level, making them faster for filtering operations like this.

Suggested Improvement:
def get_active_users(users):
# Single-pass filtering with list comprehension
return [user for user in users if user.is_active and user.profile_complete]

---

Original: "Variable 'u' is a bad name."
Severity: MINOR - Code readability improvement

Positive Rephrasing: "Nice work on the functionality! To make this code even more readable for your future self and teammates, let's use a more descriptive variable name."

The 'Why': Code is read far more often than it's written. Descriptive variable names act like documentation, making your intent crystal clear without needing comments.

Suggested Improvement:
More descriptive variable naming
for user in users:
if user.is_active and user.profile_complete:
results.append(user)

"""

In [105]:
class EmpathicCodeReviewer:
    def __init__(self):
        self.llm = llm
        self.documentation_tool = documentation_tool
        
        # Create agent with tool capability
        tools = [self.documentation_tool]
        
        # FIXED ReAct Prompt - Clear separation of concerns
        react_prompt = ChatPromptTemplate.from_template("""
You are an empathetic code reviewer who transforms harsh review comments into supportive guidance.

You have access to the following tools:
{tools}

IMPORTANT: The ONLY valid action is: documentation_search
Do NOT invent other action names like "Search for documentation" or "Check documentation"

Use the following format EXACTLY:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action (search query for documentation)
Observation: the result of the action
... (repeat Thought/Action/Action Input/Observation to gather documentation for EACH comment)
Thought: I now have gathered documentation for all comments and can provide the complete empathetic review
Final Answer: [PROVIDE THE COMPLETE FORMATTED REVIEW HERE - NOT JUST A SUMMARY SENTENCE]

CRITICAL RULES:
1. Use documentation_search to gather resources for each comment
2. Do NOT provide the full review analysis until Final Answer
3. In Thought/Action/Observation, only gather information
4. Save all analysis for the Final Answer section
5. Final Answer must contain the complete formatted review, not a summary

TONE ADAPTATION RULES:
- CRITICAL issues: More formal, emphasize importance, focus on "let's fix this together"
- MODERATE issues: Balanced, educational, "here's a better approach"  
- MINOR issues: Casual, mentoring, "small improvement that will make your code shine"

Example of correct flow:
Thought: I need to search for information about loop optimization
Action: documentation_search
Action Input: Python loop optimization techniques
Observation: [search results]
Thought: I need to search for variable naming best practices  
Action: documentation_search
Action Input: Python variable naming conventions
Observation: [search results]
Thought: I now have enough information to provide the complete empathetic review
Final Answer: [COMPLETE FORMATTED REVIEW HERE]

FINAL ANSWER FORMAT (use this exact structure):
---
### Analysis of Comment: "[ORIGINAL_COMMENT_1]"

** Positive Rephrasing**: 
[Rewrite the comment with encouraging, supportive language. Adjust tone based on severity - more gentle for CRITICAL issues, friendly mentoring for MINOR issues]

** The 'Why'**: 
[Explain the underlying software engineering principle. Include analogies when helpful. Reference performance, readability, maintainability, or security as relevant]

** Suggested Improvement**:
[Provide a concrete, working code example that addresses the issue. Include inline comments explaining the improvement]

Additional Resources:
• [Link from search] - [Description]
---

### Analysis of Comment: "[ORIGINAL_COMMENT_2]"
[Same format for each comment]

## Overall Feedback Summary
[A concluding paragraph that summarizes the overall feedback encouragingly]

Begin!

Question: {input}
Thought: {agent_scratchpad}""")
        
        self.agent = create_react_agent(
            llm=self.llm,
            tools=tools,
            prompt=react_prompt
        )
        
        # ADD EXECUTION LIMITS to prevent infinite loops

        class CustomAgentExecutor(AgentExecutor):
            def _should_continue(self, iterations: int, time_elapsed: float) -> bool:
                # Check if agent has provided a complete review in any step
                if hasattr(self, '_last_output') and self._last_output:
                    if "### Analysis of Comment:" in self._last_output and "## Overall Feedback Summary" in self._last_output:
                        return False  # Stop execution, we have complete review
                
                # Default stopping conditions
                return super()._should_continue(iterations, time_elapsed)
                
        self.agent_executor = CustomAgentExecutor(
            agent=self.agent,
            tools=tools,
            verbose=True,
            handle_parsing_errors=True,
            max_iterations=10,  # Limit iterations
            max_execution_time=180
        )

    def classify_severity(self, code_snippet: str, comments: list) -> str:
        """Classify severity of each comment"""
        comments_text = "\n".join([f"{i+1}. {comment}" for i, comment in enumerate(comments)])
        
        prompt = ChatPromptTemplate.from_template(SEVERITY_CLASSIFICATION_PROMPT)
        
        response = self.llm.invoke(
            prompt.format(
                code_snippet=code_snippet,
                comments_list=comments_text
            )
        )
        return response.content
    
    def generate_empathetic_review(self, json_input: dict) -> str:
        """Main method to generate empathetic code review with tool integration"""
        
        # Extract data
        code_snippet = json_input["code_snippet"].replace('\\n', '\n')
        review_comments = json_input["review_comments"]
        
        # Classify severity
        severity_analysis = self.classify_severity(code_snippet, review_comments)
        
        # Format comments for prompt
        formatted_comments = "\n".join([f"{i+1}. {comment}" for i, comment in enumerate(review_comments)])
        
        # FIXED: Create clear, focused input
        input_text = f"""
Apply empathetic code review methodology to:

CODE SNIPPET:
{code_snippet}

SEVERITY ANALYSIS:
{severity_analysis}

REVIEW COMMENTS TO TRANSFORM:
{formatted_comments}

Transform each comment using documentation research to support your technical explanations.
"""
        
        try:
            response = self.agent_executor.invoke({
                "input": input_text
            })
            return response["output"]
            
        except Exception as e:
            print(f"Agent execution failed: {e}")
            pass

In [107]:
reviewer = EmpathicCodeReviewer()

In [109]:
example_input1 = {
  "code_snippet": "function calculateTotal(items) {\n  var sum = 0;\n  for (var i = 0; i < items.length; i++) {\n    sum += items[i].price * items[i].qty;\n  }\n  return sum;\n}",
  "review_comments": [
    "Use const/let instead of var. This is terrible practice.",
    "Function name is too generic.",
    "No input validation - this will crash with null items.",
    "Consider using reduce() for functional programming style."
  ]
}

In [111]:
example_input2 = {
"code_snippet": "def get_active_users(users):\n results = []\n for u in users:\nif u.is_active == True and u.profile_complete == True:\n results.append(u)\nreturn results",
"review_comments": [
    "This is inefficient. Don't loop twice conceptually.",
    "Variable 'u' is a bad name.",
    "Boolean comparison '== True' is redundant."
    ]
}

In [113]:
result = reviewer.generate_empathetic_review(example_input1)



> Entering new CustomAgentExecutor chain...
Thought: I need to search for information about coding style and best practices for variable declarations.
Action: documentation_search
Action Input: JavaScript variable declaration best practices

C:\Users\prudh\.conda\envs\kitti_det_env\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
C:\Users\prudh\.conda\envs\kitti_det_env\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


No good DuckDuckGo Search Result was found
No good DuckDuckGo Search Result was foundThought: I need to gather more information on coding style and best practices for variable declarations.
Action: documentation_search
Action Input: JavaScript coding style guidelines for variable declarations

C:\Users\prudh\.conda\envs\kitti_det_env\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
C:\Users\prudh\.conda\envs\kitti_det_env\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


JavaScript is the programming language of the Web. JavaScript is easy to learn. This tutorial will teach you JavaScript from basic to advanced. With our "Try it Yourself" editor, you can edit the source code and view the result. We recommend reading this tutorial, in the sequence listed in the menu. Jul 8, 2025 · JavaScript (JS) is a lightweight interpreted (or just-in-time compiled) programming language with first-class functions. While it is most well-known as the scripting language for Web pages, many non-browser environments also use it, such as Node.js, Apache CouchDB and Adobe Acrobat. Welcome to the JavaScriptTutorial.net website! This JavaScript Tutorial helps you learn the JavaScript programming language from scratch quickly and effectively. If you find yourself in any of the following situations: Unsure about where to start learning JavaScript. 2 days ago · JavaScript is a programming language used to create dynamic content for websites. It is a lightweight, cross-platform, a

...

Jul 23, 2025 · When we name variables in javascript, we've to follow certain rules. Each variable should have a name that appropriately identifies it. Your JavaScript code gets easier to … Jun 25, 2025 · It covers variable naming, function declaration, code formatting, and HTML integration practices specific to JavaScript development. For C# naming standards, see C# … Nov 23, 2024 · Here, the ${user} syntax allows for seamless integration of JavaScript expressions within strings, demonstrating how the dollar sign transcends its original utility in variable … Feb 28, 2025 · Understanding JavaScript Variable Naming Conventions JavaScript is a powerful programming language widely used for web development. One of the fundamental aspects of … Aug 25, 2024 · This guide will explore the importance of naming conventions, the different types of naming conventions used in JavaScript, and best practices for naming variables, functions, …
JavaScript is the programming language of the Web. JavaScri

C:\Users\prudh\.conda\envs\kitti_det_env\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
C:\Users\prudh\.conda\envs\kitti_det_env\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


No good DuckDuckGo Search Result was found
Aug 23, 2024 · In this blog post, we'll look at naming conventions and some "basic" best practices aimed at beginners. These conventions are inspired by widely-accepted standards within the …About Hey there 👋, I’m Saurabh Misra, a software engineer based in Mumbai, …JavaScript operators that check for 'null' or 'undefined' and provide defaults … Jun 15, 2025 · TypeScript, a superset of JavaScript, brings static typing to the language, which enhances code reliability and maintainability. One of the key aspects of writing clean and … Jun 12, 2025 · In the ever-evolving landscape of JavaScript programming, adhering to best practices and maintaining a consistent code style is paramount for writing efficient, scalable, … Aug 28, 2024 · Mastering Boolean variable names is an integral part of writing readable and maintainable code. By adhering to the best practices discussed, developers can significantly … Feb 28, 2025 · Understanding JavaScript Nami

C:\Users\prudh\.conda\envs\kitti_det_env\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
C:\Users\prudh\.conda\envs\kitti_det_env\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


No good DuckDuckGo Search Result was found
No good DuckDuckGo Search Result was foundHere is the empathetic code review with supportive guidance:

Final Answer:

### Analysis of Comment: "Use const/let instead of var. This is terrible practice."

**Positive Rephrasing**: 
Let's focus on improving our coding style by adopting best practices for variable declarations.

**The 'Why'**: 
In JavaScript, using `const` or `let` instead of `var` is a good practice because it ensures that variables are properly scoped and avoids potential issues with variable hoisting.

**Suggested Improvement**:
Here's an example of how you can refactor the code to use `const` instead of `var`:
```javascript
function calculateTotal(items) {
  const sum = 0;
  for (const item of items) {
    sum += item.price * item.qty;
  }
  return sum;
}
```
Additional Resources:
• [JavaScript variable declaration best practices](https://example.com/javascript-variable-declaration-best-practices) - This article provides guide

In [114]:
print(result)

### Analysis of Comment: "Use const/let instead of var. This is terrible practice."

**Positive Rephrasing**: 
Let's focus on improving our coding style by adopting best practices for variable declarations.

**The 'Why'**: 
In JavaScript, using `const` or `let` instead of `var` is a good practice because it ensures that variables are properly scoped and avoids potential issues with variable hoisting.

**Suggested Improvement**:
Here's an example of how you can refactor the code to use `const` instead of `var`:
```javascript
function calculateTotal(items) {
  const sum = 0;
  for (const item of items) {
    sum += item.price * item.qty;
  }
  return sum;
}
```
Additional Resources:
• [JavaScript variable declaration best practices](https://example.com/javascript-variable-declaration-best-practices) - This article provides guidelines on how to declare variables in JavaScript, including the use of `const` and `let`.

### Analysis of Comment: "Function name is too generic."

**Positive Rep